In [2]:
import numpy as np
import tensorflow as tf
import keras
import cv2
from keras.layers import MaxPool2D,Conv2D,UpSampling2D,Input,Dropout
from keras.models import Sequential
from keras.preprocessing.image import img_to_array
import os
from tqdm import tqdm
import re
import matplotlib.pyplot as plt

In [3]:

def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

SIZE = 224
color_img = []
path = '../input/landscape-image-colorization/landscape Images/color'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):    
    if i == '1000.jpg':
        break
    else:    
        img = cv2.imread(path + '/'+i,1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0
        color_img.append(img_to_array(img))


gray_img = []
path = '../input/landscape-image-colorization/landscape Images/gray'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
     if i == '1000.jpg':
        break
     else: 
        img = cv2.imread(path + '/'+i,1)

   
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0
        gray_img.append(img_to_array(img))


In [4]:
train_gray_image = gray_img[:1000]
train_color_image = color_img[:1000]


train_g = np.reshape(train_gray_image,(len(train_gray_image),SIZE,SIZE,3))
train_c = np.reshape(train_color_image, (len(train_color_image),SIZE,SIZE,3))


In [5]:
train_g.shape

In [6]:
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import keras
import os

from keras.applications.vgg16 import VGG16
vggmodel = VGG16()
newmodel = Sequential() 

for i, layer in enumerate(vggmodel.layers):
    if i<19:         
      newmodel.add(layer)
newmodel.summary()
for layer in newmodel.layers:
  layer.trainable=False  


In [7]:
vggfeatures = []




In [8]:
prediction = newmodel.predict(train_g)
prediction = prediction.reshape(1000,7,7,512)
vggfeatures.append(prediction)

vggfeatures = np.array(vggfeatures)

print(vggfeatures.shape)

In [9]:
vggfeatures = vggfeatures.reshape(1000,7,7,512)

In [10]:
vggfeatures.shape

In [11]:
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()


model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, train_c, verbose=1, epochs=1000, batch_size=128)

In [36]:
model.save('colorize_autoencoder_VGG16.model')

In [13]:
from keras.models import load_model
model1 = load_model('colorize_autoencoder_VGG16.model',
                                   custom_objects=None,
                                   compile=True)

In [29]:
i1 = gray_img[4]

In [30]:
i1.shape

In [31]:
plt.imshow(i1)

In [32]:
i1 = i1.reshape(1,224,224,3)

In [33]:
vgg_pred1 = newmodel.predict(i1)
img_pred1 = model.predict(vgg_pred1)

In [34]:
img_pred1 = img_pred1.reshape(224,224,3)

In [35]:
plt.imshow(img_pred1)